# Lesson 15 - use antibiotics wisely!

The UK Health Security Agency developed the UK Access, Watch, Reserve, and Other classification for antibiotics (UK-AWaRe). This classification holds four categories:

1. **Access antibiotics** are generally narrow-spectrum and associated with a lower risk of resistance. They are recommended as first or second choice options for common infections and should be readily available.
2. **Watch antibiotics** are broader-spectrum with a higher resistance potential. Their use should be limited to specific indications and monitored closely.
3. **Reserve antibiotics** are last-line treatments, often reserved for severe infections caused by multidrug-resistant organisms. These medicines require strict oversight to maintain their effectiveness.
4. **Other**

It is useful to see what fraction of all prescriptions are from the **watch** and **reserve** groups, as these should be used cautiously to reduce antibiotic resistence and hence effectiveness.

You can see the results of the Bennett analysis of the UK-AWaRe [here](https://openprescribing.net/measure/aware_antibiotics/).

Let's have a go at getting the data for the above ask ourselves and plotting it.

## Imports and Path objects

You should be happy with imports and Path objects now:

In [ ]:
from pathlib import Path
from ebmdatalab import bq

DATA_FOLDER = Path("data")

And here we send off the SQL query and save the results in a csv file.

In [ ]:
sql = """
-- Get the month, aware (access, watch and reserve) and sum of items
SELECT month, aware, SUM(items) AS items

-- Use the below database table
FROM measures.vw__aware_abx

-- Pick out only rows where aware is either Access, Watch or Reserve
WHERE aware IN ('Access','Watch','Reserve')

-- We group our results first by month and then a sub-group of aware value
GROUP BY month, aware

-- We finally order by month and then aware
ORDER BY month, aware
"""

raw = bq.cached_read(sql, DATA_FOLDER / "aware_raw.csv", use_cache=True)
                    
raw

## Let's clean up those dates

The date and time values `2010-08-01 00:00:00+00:00` look a little messy, let's get ride of the time value:

In [ ]:
import pandas as pd

raw["month"] = pd.to_datetime(raw["month"], utc=True).dt.tz_convert(None).dt.date

raw

## Go wide!

This bit is magical, well almost. What we do here is 3-staged:
```python
raw.pivot(index="month", columns="aware", values="items")
```
In the above code, we take the `raw` data, cluster all of the `aware` values associated with the same date and put them in new columns

```python
.fillna(0)
```

This bit of code fills in data that is missing with a zero `0`.

```python
.reset_index()
```

This last bit of code puts the month in its own column (useful for calculations later).

Putting the data in this `wide format` helps us prepare the data to then plot it out.

In [ ]:
wide_format = (
    raw.pivot(index="month", columns="aware", values="items")
    .fillna(0)
    .reset_index()
)

wide_format

## Hold the phone!

**Now just to make things** more interesting, and also to fit everything into the standard screen width, we have used parentheses `( )` to enable us to place the code over several lines instead of just one line. We will do this in a few places to keep you on your toes! But trust me, it is for your own good!

## Numerator

Let's calculate the numerators. As you can see this is done by adding the sum of `Watch` and `Reserve` items.

In [ ]:
wide_format["numerator_items"] = wide_format["Watch"] + wide_format["Reserve"]

## Denominator

Let's calculate the denominators. As you can see this is the sum of all three aware types that we have at hand (ie `Access`, `Watch` and `Reserve`).

In [ ]:
wide_format["denominator_items"] = (
    wide_format["Access"] 
    + wide_format["Watch"] 
    + wide_format["Reserve"]
)

## Fraction and the percentages

Here we are working out fractions:

```python
wide_format["numerator_items"]
    .div(wide_format["denominator_items"])
```

And then multiplying by 100 to get a percentage:

```python
.mul(100)
```

And then rounding up to 2 decimal points

In [ ]:
wide_format["percent_watch_aware"] = (
    wide_format["numerator_items"]
    .div(wide_format["denominator_items"])
    .mul(100)
    .round(1)
)

wide_format

## Clean it up!

Let's remove all of the columns we no longer need as we have calculated what we need to next plot out the data. We do this by stating what columns we need in square brackets within square brackets.

In [ ]:
final_data = wide_format[["month", "percent_watch_aware"]]

final_data

## Show us the goods!

Next we ask the kind `matplotlib` library to make a plot of our data.

In [ ]:
import matplotlib.pyplot as plt

# We create an `axes` object to put out data into
# This is a slightly different way to plot our data than in lesson 14, but it still works!
ax = final_data.plot(
    x="month",
    y="percent_watch_aware",
    kind="line",
    legend=False
)

# We set out x and y axis labels
ax.set_ylabel("Percentage of Watch & Reserve")
ax.set_xlabel("Time")

# We add a nice title
ax.set_title("AWaRe stewardship over time")

# Plot it!
plt.show()